# Project 1: Image Feature Extraction and Matching

This is Project 1 for [UW CSE P576 Computer Vision](https://courses.cs.washington.edu/courses/csep576/18sp). 

**Getting Started:** To get started, **[download the source files here (Projects 1 and 2)](https://courses.cs.washington.edu/courses/csep576/18sp/projects/project12/project12.zip "Project 1 and 2 Source Files")**. To run the project locally you will need IPython/Jupyter installed, e.g., see instructions at http://jupyter.org/install.html. Launch Jupyter and open `Project1.ipynb`. Alternatively, you can import the standalone version of the notebook into [Colaboratory](https://colab.research.google.com "Colab") and run it without installing anything. Use File->Upload Notebook in Colab and open the notebook in `standalone/Project1s.ipynb`.

**This project:** In this project you will build an image feature matcher, starting with simple convolution operations and working through interest point detection and descriptor extraction. Once you have a basic feature matcher working, try out some improvements and document your results. If you’re not already familiar with python/numpy, it is recommended to do an introduction such as: http://cs231n.github.io/python-numpy-tutorial. 

**What to turn in:** Turn in a pdf or static html copy of your completed ipynb notebook as well as the source .ipynb and any source .py files that you modified. Clearly describe any enhancements or experiments you tried in your ipynb notebook.

In [ ]:
import numpy as np
import os.path
from time import time
import types
import matplotlib.pyplot as plt

#import im_util
#import interest_point

%matplotlib inline
# edit this line to change the figure size
plt.rcParams['figure.figsize'] = (16.0, 10.0)
# force auto-reload of import modules before running code 
%load_ext autoreload
%autoreload 2

In [ ]:
!wget -nc https://courses.cs.washington.edu/courses/csep576/18sp/projects/project12/pano_images.zip && unzip -n -d data pano_images.zip

### im_util.py

In [ ]:
# Copyright 2017 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import PIL.Image as pil
import scipy.signal as sps
import matplotlib.pyplot as plt
from scipy.ndimage import map_coordinates

def convolve_1d(x, k):
  """
  Convolve vector x with kernel k

  Inputs: x=input vector (Nx)
          k=input kernel (Nk)

  Outputs: y=output vector (Nx)
  """
  y=np.zeros_like(x)

  """
  *******************************************
  *** TODO: write code to perform convolution
  *******************************************

  The output should be the same size as the input
  You can assume zero padding, and an odd-sized kernel
  """


  """
  *******************************************
  """

  return y

def convolve_rows(im, k):
  """
  Convolve image im with kernel k

  Inputs: im=input image (H, W, B)
          k=1D convolution kernel (N)

  Outputs: im_out=output image (H, W, B)
  """
  im_out = np.zeros_like(im)

  """
  *****************************************
  *** TODO: write code to convolve an image
  *****************************************

  Convolve the rows of image im with kernel k
  The output should be the same size as the input
  You can assume zero padding, and an odd-sized kernel
  """


  """
  *****************************************
  """

  return im_out

def gauss_kernel(sigma):
  """
  1D Gauss kernel of standard deviation sigma
  """
  l = int(np.ceil(2 * sigma))
  x = np.linspace(-l, l, 2*l+1)

  # FORNOW
  gx = np.zeros_like(x)

  """
  *******************************************
  *** TODO: compute gaussian kernel at each x
  *******************************************
  """


  """
  *******************************************
  """

  gx = np.expand_dims(gx,0)
  return gx

def convolve_gaussian(im, sigma):
  """
  2D gaussian convolution
  """
  imc=np.zeros_like(im)

  """
  ***************************************
  *** TODO separable gaussian convolution
  ***************************************
  """


  """
  ***************************************
  """
  return imc

def compute_gradients(img):

  Ix=np.zeros_like(img)
  Iy=np.zeros_like(img)

  """
  ***********************************************
  *** TODO: write code to compute image gradients
  ***********************************************
  """


  """
  ***********************************************
  """
  return Ix, Iy

def image_open(filename):
  """
  Returns a numpy float image with values in the range (0,1)
  """
  pil_im = pil.open(filename)
  im_np = np.array(pil_im).astype(np.float32)
  im_np /= 255.0
  return im_np

def image_save(im_np, filename):
  """
  Saves a numpy float image to file
  """
  if (len(im_np.shape)==2):
    im_np = np.expand_dims(im_np, 2)
  if (im_np.shape[2]==1):
    im_np= np.repeat(im_np, 3, axis=2)
  im_np = np.maximum(0.0, np.minimum(im_np, 1.0))
  pil_im = pil.fromarray((im_np*255).astype(np.uint8))
  pil_im.save(filename)

def image_figure(im, dpi=100):
  """
  Creates a matplotlib figure around an image,
  useful for writing to file with savefig()
  """
  H,W,_=im.shape
  fig=plt.figure()
  fig.set_size_inches(W/dpi, H/dpi)
  ax=fig.add_axes([0,0,1,1])
  ax.imshow(im)
  return fig, ax

def plot_two_images(im1, im2):
  """
  Plot two images and return axis handles
  """
  ax1=plt.subplot(1,2,1)
  plt.imshow(im1)
  plt.axis('off')
  ax2=plt.subplot(1,2,2)
  plt.imshow(im2)
  plt.axis('off')
  return ax1, ax2

def normalise_01(im):
  """
  Normalise image to the range (0,1)
  """
  mx = im.max()
  mn = im.min()
  den = mx-mn
  small_val = 1e-9
  if (den < small_val):
    print('image normalise_01 -- divisor is very small')
    den = small_val
  return (im-mn)/den

def grey_to_rgb(img):
  """
  Convert greyscale to rgb image
  """
  if (len(img.shape)==2):
    img = np.expand_dims(img, 2)

  img3 = np.repeat(img, 3, 2)
  return img3

def disc_mask(l):
  """
  Create a binary cirular mask of radius l
  """
  sz = 2 * l + 1
  m = np.zeros((sz,sz))
  x = np.linspace(-l,l,2*l+1)/l
  x = np.expand_dims(x, 1)
  m = x**2
  m = m + m.T
  m = m<1
  m = np.expand_dims(m, 2)
  return m

def convolve(im, kernel):
  """
  Wrapper for scipy convolution function
  This implements a general 2D convolution of image im with kernel
  Note that strictly speaking this is correlation not convolution

  Inputs: im=input image (H, W, B) or (H, W)
          kernel=kernel (kH, kW)

  Outputs: imc=output image (H, W, B)
  """
  if (len(im.shape)==2):
    im = np.expand_dims(im, 2)
  H, W, B = im.shape
  imc = np.zeros((H, W, B))
  for band in range(B):
    imc[:, :, band] = sps.correlate2d(im[:, :, band], kernel, mode='same')
  return imc

def coordinate_image(num_rows,num_cols,r0,r1,c0,c1):
  """
  Creates an image size num_rows, num_cols
  with coordinates linearly spaced in from r0->r1 and c0->c1
  """
  rval=np.linspace(r0,r1,num_rows)
  cval=np.linspace(c0,c1,num_cols)
  c,r=np.meshgrid(cval,rval)
  M = np.stack([r,c,np.ones(r.shape)],-1)
  return M

def transform_coordinates(coord_image, M):
  """
  Transform an image containing row,col,1 coordinates by matrix M
  """
  M=np.expand_dims(M,2)
  uh=np.dot(coord_image,M.T)
  uh=uh[:, :, 0, :]
  uh=uh/np.expand_dims(uh[:, :, 2],2)
  return uh

def warp_image(im, coords):
  """
  Warp image im using row,col,1 image coords
  """
  im_rows,im_cols,im_bands=im.shape
  warp_rows,warp_cols,_=coords.shape
  map_coords=np.zeros((3,warp_rows,warp_cols,im_bands))
  for b in range(im_bands):
    map_coords[0,:,:,b]=coords[:,:,0]
    map_coords[1,:,:,b]=coords[:,:,1]
    map_coords[2,:,:,b]=b
  warp_im = map_coordinates(im, map_coords, order=1)
  return warp_im

# allow accessing these functions by im_util.*
im_util=types.SimpleNamespace()
im_util.convolve_1d=convolve_1d
im_util.convolve_rows=convolve_rows
im_util.gauss_kernel=gauss_kernel
im_util.convolve_gaussian=convolve_gaussian
im_util.compute_gradients=compute_gradients
im_util.image_open=image_open
im_util.image_save=image_save
im_util.image_figure=image_figure
im_util.plot_two_images=plot_two_images
im_util.normalise_01=normalise_01
im_util.grey_to_rgb=grey_to_rgb
im_util.disc_mask=disc_mask
im_util.convolve=convolve
im_util.coordinate_image=coordinate_image
im_util.transform_coordinates=transform_coordinates
im_util.warp_image=warp_image

### interest_point.py

In [ ]:
# Copyright 2017 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import scipy.ndimage.filters as filters
from scipy.ndimage import map_coordinates
from matplotlib.patches import Circle
import matplotlib.pyplot as plt

#import im_util

class InterestPointExtractor:
  """
  Class to extract interest points from an image
  """
  def __init__(self):
    self.params={}
    self.params['border_pixels']=10
    self.params['strength_threshold_percentile']=95
    self.params['supression_radius_frac']=0.01

  def find_interest_points(self, img):
    """
    Find interest points in greyscale image img

    Inputs: img=greyscale input image (H, W, 1)

    Outputs: ip=interest points of shape (2, N)
    """
    ip_fun = self.corner_function(img)
    row, col = self.find_local_maxima(ip_fun)

    ip = np.stack((row,col))
    return ip

  def corner_function(self, img):
    """
    Compute corner strength function in image im

    Inputs: img=grayscale input image (H, W, 1)

    Outputs: ip_fun=interest point strength function (H, W, 1)
    """

    H, W, _ = img.shape

    # FORNOW: random interest point function
    ip_fun = np.random.randn(H, W, 1)

    """
    **********************************************************
    *** TODO: write code to compute a corner strength function
    **********************************************************
    """


    """
    **********************************************************
    """

    return ip_fun

  def find_local_maxima(self, ip_fun):
    """
    Find local maxima in interest point strength function

    Inputs: ip_fun=corner strength function (H, W, 1)

    Outputs: row,col=coordinates of interest points
    """

    H, W, _ = ip_fun.shape

    # radius for non-maximal suppression
    suppression_radius_pixels = int(self.params['supression_radius_frac']*max(H, W))

    # minimum of strength function for corners
    strength_threshold=np.percentile(ip_fun, self.params['strength_threshold_percentile'])

    # don't return interest points within border_pixels of edge
    border_pixels = self.params['border_pixels']

    # row and column coordinates of interest points
    row = []
    col = []

    # FORNOW: random row and column coordinates
    row = np.random.randint(0,H,100)
    col = np.random.randint(0,W,100)

    """
    ***************************************************
    *** TODO: write code to find local maxima in ip_fun
    ***************************************************

    Hint: try scipy filters.maximum_filter with im_util.disc_mask
    """


    """
    ***************************************************
    """

    return row, col

class DescriptorExtractor:
  """
  Extract descriptors around interest points
  """
  def __init__(self):
    self.params={}
    self.params['patch_size']=8
    self.params['ratio_threshold']=1.0

  def get_descriptors(self, img, ip):
    """
    Extact descriptors from grayscale image img at interest points ip

    Inputs: img=grayscale input image (H, W, 1)
            ip=interest point coordinates (2, N)

    Returns: descriptors=vectorized descriptors (N, num_dims)
    """
    patch_size=self.params['patch_size']
    patch_size_div2=int(patch_size/2)
    num_dims=patch_size**2

    H,W,_=img.shape
    num_ip=ip.shape[1]
    descriptors=np.zeros((num_ip,num_dims))


    for i in range(num_ip):
      row=ip[0,i]
      col=ip[1,i]

      # FORNOW: random image patch
      patch=np.random.randn(patch_size,patch_size)

      """
      ******************************************************
      *** TODO: write code to extract descriptor at row, col
      ******************************************************
      """


      """
      ******************************************************
      """

      descriptors[i, :]=np.reshape(patch,num_dims)

    # normalise descriptors to 0 mean, unit length
    mn=np.mean(descriptors,1,keepdims=True)
    sd=np.std(descriptors,1,keepdims=True)
    small_val = 1e-6
    descriptors = (descriptors-mn)/(sd+small_val)

    return descriptors

  def compute_distances(self, desc1, desc2):
    """
    Compute distances between descriptors

    Inputs: desc1=descriptor array (N1, num_dims)
            desc2=descriptor array (N2, num_dims)

    Returns: dists=array of distances (N1,N2)
    """
    N1,num_dims=desc1.shape
    N2,num_dims=desc2.shape

    ATB=np.dot(desc1,desc2.T)
    AA=np.sum(desc1*desc1,1)
    BB=np.sum(desc2*desc2,1)

    dists=-2*ATB+np.expand_dims(AA,1)+BB

    return dists

  def match_descriptors(self, desc1, desc2):
    """
    Find nearest neighbour matches between descriptors

    Inputs: desc1=descriptor array (N1, num_dims)
            desc2=descriptor array (N2, num_dims)

    Returns: match_idx=nearest neighbour index for each desc1 (N1)
    """
    dists=self.compute_distances(desc1, desc2)

    match_idx=np.argmin(dists,1)

    return match_idx

  def match_ratio_test(self, desc1, desc2):
    """
    Find nearest neighbour matches between descriptors
    and perform ratio test

    Inputs: desc1=descriptor array (N1, num_dims)
            desc2=descriptor array (N2, num_dims)

    Returns: match_idx=nearest neighbour inded for each desc1 (N1)
             ratio_pass=whether each match passes ratio test (N1)
    """
    N1,num_dims=desc1.shape

    dists=self.compute_distances(desc1, desc2)

    sort_idx=np.argsort(dists,1)

    #match_idx=np.argmin(dists,1)
    match_idx=sort_idx[:,0]

    d1NN=dists[np.arange(0,N1),sort_idx[:,0]]
    d2NN=dists[np.arange(0,N1),sort_idx[:,1]]

    ratio_threshold=self.params['ratio_threshold']
    ratio_pass=(d1NN<ratio_threshold*d2NN)

    return match_idx,ratio_pass

def draw_interest_points_ax(ip, ax):
  """
  Draw interest points ip on axis ax
  """
  for row,col in zip(ip[0,:],ip[1,:]):
    circ1 = Circle((col,row), 5)
    circ1.set_color('black')
    circ2 = Circle((col,row), 3)
    circ2.set_color('white')
    ax.add_patch(circ1)
    ax.add_patch(circ2)

def draw_interest_points_file(im, ip, filename):
  """
  Draw interest points ip on image im and save to filename
  """
  fig,ax = im_util.image_figure(im)
  draw_interest_points_ax(ip, ax)
  fig.savefig(filename)
  plt.close(fig)

def draw_matches_ax(ip1, ipm, ax1, ax2):
  """
  Draw matches ip1, ipm on axes ax1, ax2
  """
  for r1,c1,r2,c2 in zip(ip1[0,:], ip1[1,:], ipm[0,:], ipm[1,:]):
    rand_colour=np.random.rand(3,)

    circ1 = Circle((c1,r1), 5)
    circ1.set_color('black')
    circ2 = Circle((c1,r1), 3)
    circ2.set_color(rand_colour)
    ax1.add_patch(circ1)
    ax1.add_patch(circ2)

    circ3 = Circle((c2,r2), 5)
    circ3.set_color('black')
    circ4 = Circle((c2,r2), 3)
    circ4.set_color(rand_colour)
    ax2.add_patch(circ3)
    ax2.add_patch(circ4)

def draw_matches_file(im1, im2, ip1, ipm, filename):
  """
  Draw matches ip1, ipm on images im1, im2 and save to filename
  """
  H1,W1,B1=im1.shape
  H2,W2,B2=im2.shape

  im3 = np.zeros((max(H1,H2),W1+W2,3))
  im3[0:H1,0:W1,:]=im1
  im3[0:H2,W1:(W1+W2),:]=im2

  fig,ax = im_util.image_figure(im3)
  col_offset=W1

  for r1,c1,r2,c2 in zip(ip1[0,:], ip1[1,:], ipm[0,:], ipm[1,:]):
    rand_colour=np.random.rand(3,)

    circ1 = Circle((c1,r1), 5)
    circ1.set_color('black')
    circ2 = Circle((c1,r1), 3)
    circ2.set_color(rand_colour)
    ax.add_patch(circ1)
    ax.add_patch(circ2)

    circ3 = Circle((c2+col_offset,r2), 5)
    circ3.set_color('black')
    circ4 = Circle((c2+col_offset,r2), 3)
    circ4.set_color(rand_colour)
    ax.add_patch(circ3)
    ax.add_patch(circ4)

  fig.savefig(filename)
  plt.close(fig)

def plot_descriptors(desc,plt):
  """
  Plot a random set of descriptor patches
  """
  num_ip,num_dims = desc.shape
  patch_size = int(np.sqrt(num_dims))

  N1,N2=2,8
  figsize0=plt.rcParams['figure.figsize']
  plt.rcParams['figure.figsize'] = (16.0, 4.0)
  for i in range(N1):
    for j in range(N2):
      ax=plt.subplot(N1,N2,i*N2+j+1)
      rnd=np.random.randint(0,num_ip)
      desc_im=np.reshape(desc[rnd,:],(patch_size,patch_size))
      plt.imshow(im_util.grey_to_rgb(im_util.normalise_01(desc_im)))
      plt.axis('off')

  plt.rcParams['figure.figsize']=figsize0

def plot_matching_descriptors(desc1,desc2,desc1_id,desc2_id,plt):
  """
  Plot a random set of matching descriptor patches
  """
  num_inliers=desc1_id.size
  num_ip,num_dims = desc1.shape
  patch_size=int(np.sqrt(num_dims))

  figsize0=plt.rcParams['figure.figsize']

  N1,N2=1,8
  plt.rcParams['figure.figsize'] = (16.0, N1*4.0)

  for i in range(N1):
    for j in range(N2):
      rnd=np.random.randint(0,num_inliers)

      desc1_rnd=desc1_id[rnd]
      desc2_rnd=desc2_id[rnd]

      desc1_im=np.reshape(desc1[desc1_rnd,:],(patch_size,patch_size))
      desc2_im=np.reshape(desc2[desc2_rnd,:],(patch_size,patch_size))

      ax=plt.subplot(2*N1,N2,2*i*N2+j+1)
      plt.imshow(im_util.grey_to_rgb(im_util.normalise_01(desc1_im)))
      plt.axis('off')
      ax=plt.subplot(2*N1,N2,2*i*N2+N2+j+1)
      plt.imshow(im_util.grey_to_rgb(im_util.normalise_01(desc2_im)))
      plt.axis('off')

  plt.rcParams['figure.figsize'] = figsize0

# allow accessing these functions by interest_point.*
interest_point=types.SimpleNamespace()
interest_point.InterestPointExtractor=InterestPointExtractor
interest_point.DescriptorExtractor=DescriptorExtractor
interest_point.draw_interest_points_ax=draw_interest_points_ax
interest_point.draw_interest_points_file=draw_interest_points_file
interest_point.draw_matches_ax=draw_matches_ax
interest_point.draw_matches_file=draw_matches_file
interest_point.plot_descriptors=plot_descriptors
interest_point.plot_matching_descriptors=plot_matching_descriptors

### Convolution and Image Filtering [25%]

Start by writing code to perform convolution in 1D. Open `im_util.py` and edit the function `convolve_1d`. You should use only basic numpy array operations and loops. Don't worry about efficiency for now. You should see small errors compared to the reference numpy version.

Note that convolution and correlation are the same under a simple manipulation of the kernel (what is it?). For what kernels are convolution and correlation results identical?

In [ ]:
"""
Test of convolve_1d
"""
print('[ Test convolve_1d ]')
x = (np.random.rand(20)>0.8).astype(np.float32)
k = np.array([1,3,1])
y1 = im_util.convolve_1d(x, k)
y2 = np.convolve(x, k, 'same')
y3 = np.correlate(x, k, 'same')
print(' convolve error = ', np.sum((y1-y2)**2))
print(' correlate error = ', np.sum((y1-y3)**2))

We will now convolve a 2D image with a 1D kernel. Before you begin, get some image data by running `get_data.sh` in the `data` directory. Then complete the function `convolve_rows` in `im_util.py` by convolving every row of the image by the kernel. Run the code below and check that the image output is sensible.

In [ ]:
"""
Test of convolve_image
"""
image_filename='data/test/100-0038_img.jpg'

print('[ Test convolve_image ]')
im = im_util.image_open(image_filename)
k = np.array([1,2,3,4,5,6,5,4,3,2,1])
print(' convolve_rows')
t0=time()
im1 = im_util.convolve_rows(im, k)
t1=time()
print(' % .2f secs' % (t1-t0))
print(' scipy convolve')
t0=time()
im2 = im_util.convolve(im, np.expand_dims(k,0))
t1=time()
print(' % .2f secs' % (t1-t0))
print(' convolve_image error =', np.sum((im1-im2)**2))

# optionally plot images for debugging
#im1_norm=im_util.normalise_01(im1)
#im2_norm=im_util.normalise_01(im2)
#ax1,ax2=im_util.plot_two_images(im1_norm, im2_norm)          

You will probably find that the scipy convolve runs much faster than your version. To speed things up you can use this version (`im_util.convolve`) for all subsequent experiments. Note that this performs a general 2D convolution with a 2D kernel as input. 

Now write code to perform Gaussian blurring. First implement the function `gauss_kernel` to compute a 1D Gaussian kernel. Then complete `convolve_gaussian` to perform a separable convolution with this kernel.

In [ ]:
"""
Gaussian blurring test
"""
print('[ Test convolve_gaussian ]')

sigma=4.0
k=im_util.gauss_kernel(sigma)
print(' gauss kernel = ')
print(k)

t0=time()
im1 = im_util.convolve_gaussian(im, sigma)
t1=time()
print(' % .2f secs' % (t1-t0))

ax1,ax2=im_util.plot_two_images(im, im1)

Now write code to compute horizontal and vertical gradients in the function `compute_gradients`. Use an explicit kernel that is convolved in each direction (i.e., do not use a built-in function such as `numpy.gradient`). Run the code below and check that the outputs look sensible.

In [ ]:
"""
Gradient computation test
"""
print('[ Test gradient computation ]')
img = np.mean(im,2,keepdims=True)
Ix,Iy = im_util.compute_gradients(img)

# copy greyvalue to RGB channels
Ix_out = im_util.grey_to_rgb(im_util.normalise_01(Ix))
Iy_out = im_util.grey_to_rgb(im_util.normalise_01(Iy))

im_util.plot_two_images(Ix_out, Iy_out)

### Interest Point Extractor [25%]

Now you will use these convolution functions to implement a corner or interest point detector. Choose a well known detector, such as Harris or DoG, and implement the interest point strength function in `corner_function` of `interest_point.py`. Run the code below to visualise your corner function output. Next detect corners as local maxima of this function by filling in `find_local_maxima` in the same file.

In [ ]:
"""
Compute corner strength function
"""
print('[ Compute corner strength ]')
ip_ex = interest_point.InterestPointExtractor()
ip_fun = ip_ex.corner_function(img)

# normalise for display
[mn,mx]=np.percentile(ip_fun,[5,95])
small_val=1e-9
ip_fun_norm=(ip_fun-mn)/(mx-mn+small_val)
ip_fun_norm=np.maximum(np.minimum(ip_fun_norm,1.0),0.0)

"""
Find local maxima of corner strength
"""
print('[ Find local maxima ]')
row, col = ip_ex.find_local_maxima(ip_fun)
ip = np.stack((row,col))

ax1,ax2=im_util.plot_two_images(im_util.grey_to_rgb(ip_fun_norm),im)
interest_point.draw_interest_points_ax(ip, ax2)

### Descriptors and Matching [25%]

Now let's match our interest points. Start by extracting a very simple descriptor that is simply a patch of pixels around the interest point. To do this, fill in the function `get_descriptors` in `interest_point.py`. The following code outputs a random sample of normalised descriptor patches. Check that the output looks sensible. Once you have this working, try varying the sample spacing in your descriptor patch. What problem exists with sample spacings > 1 pixel? How can this be fixed?

In [ ]:
"""
Extract descriptors
"""
print('[ Extract descriptors ]')
desc_ex=interest_point.DescriptorExtractor()
descriptors=desc_ex.get_descriptors(img, ip)
interest_point.plot_descriptors(descriptors,plt)

We will now match descriptors between a pair of images. Run the following two code blocks to extract your interest points and extract and match descriptors. The second code block calls a function to perform nearest-neighbour matching of descriptors and filtering using a ratio test. Take a look at the code and check you understand how it works.

In [ ]:
"""
Read a pair of input images and extract interest points
"""
image_dir='data/test'
#im_filename1=image_dir+'/100-0023_img.jpg'
#im_filename2=image_dir+'/100-0024_img.jpg'
im_filename1=image_dir+'/100-0038_img.jpg'
im_filename2=image_dir+'/100-0039_img.jpg'

im1 = im_util.image_open(im_filename1)
im2 = im_util.image_open(im_filename2)

img1 = np.mean(im1, 2, keepdims=True)
img2 = np.mean(im2, 2, keepdims=True)

print('[ find interest points ]')
t0=time()
ip_ex = interest_point.InterestPointExtractor()
ip1 = ip_ex.find_interest_points(img1)
print(' found '+str(ip1.shape[1])+' in image 1')
ip2 = ip_ex.find_interest_points(img2)
print(' found '+str(ip2.shape[1])+' in image 2')
t1=time()
print(' % .2f secs ' % (t1-t0))

print('[ drawing interest points ]')
ax1,ax2=im_util.plot_two_images(im1,im2)
t0=time()
interest_point.draw_interest_points_ax(ip1, ax1)
interest_point.draw_interest_points_ax(ip2, ax2)
t1=time()
print(' % .2f secs ' % (t1-t0))

In [ ]:
"""
Extract and match descriptors
"""
print('[ extract descriptors ]')
t0=time()
desc_ex = interest_point.DescriptorExtractor()
desc1 = desc_ex.get_descriptors(img1, ip1)
desc2 = desc_ex.get_descriptors(img2, ip2)
t1=time()
print(' % .2f secs' % (t1-t0))

print('[ match descriptors ]')
match_idx,ratio_pass=desc_ex.match_ratio_test(desc1, desc2)
num_ratio_pass=np.sum(ratio_pass)

ipm=ip2[:,match_idx]

ip1r=ip1[:,ratio_pass]
ip2r=ipm[:,ratio_pass]

N1,num_dims=desc1.shape
print(' Number of interest points = '+str(N1))
print(' Number of matches passing ratio test = '+str(num_ratio_pass))

ax1,ax2=im_util.plot_two_images(im1,im2)
interest_point.draw_matches_ax(ip1r, ip2r, ax1, ax2)


The following code visualises matched descriptor patches. Can you distinguish the correct and incorrect matches? (reload to get another random sample).

In [ ]:
"""
Plot descriptors for matched points
"""
interest_point.plot_matching_descriptors(desc1,desc2,np.arange(0,ip1.shape[1]),match_idx,plt)

### Testing and Improving Feature Matching [25%]

Try varying the `ratio_threshold` parameter in the descriptor matcher (`DescriptorExtractor` class params). What are good settings for this parameter? If everything is working, you should see a good set of correctly matched points (aim for about 100 or more). Experiment with your interest point and descriptor implementations to find which parameters are important and try to get a good set of matches. Try out a new idea of your own to improve interest points or descriptors, and record your findings in the notebook below.

In [ ]:
### TODO experiments with your detector/descriptors
